In [1]:
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
import requests
import numpy as np
import pandas as pd
import io
import pickle

In [2]:
# Import the preprocessed data
X_preprocessed_url = "https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_2/datasets/pickles/preprocessed_fungi.pkl"
y_url = "https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_2/datasets/pickles/fungi_y.pkl"

X = pickle.load(io.BytesIO(requests.get(X_preprocessed_url).content))
y = pickle.load(io.BytesIO(requests.get(y_url).content))

print(X[0])
print(y.head())

[[[0.5568628  0.5647059  0.56078434]
  [0.5568628  0.5647059  0.56078434]
  [0.5568628  0.5647059  0.56078434]
  ...
  [0.53333336 0.54901963 0.54509807]
  [0.5372549  0.5529412  0.54901963]
  [0.5372549  0.5529412  0.54901963]]

 [[0.54901963 0.5568628  0.5529412 ]
  [0.54901963 0.5568628  0.5529412 ]
  [0.5529412  0.56078434 0.5568628 ]
  ...
  [0.53333336 0.54901963 0.54509807]
  [0.5372549  0.5529412  0.54901963]
  [0.5372549  0.5529412  0.54901963]]

 [[0.53333336 0.5411765  0.5372549 ]
  [0.5411765  0.54901963 0.54509807]
  [0.54901963 0.5568628  0.5529412 ]
  ...
  [0.5254902  0.5411765  0.5372549 ]
  [0.5294118  0.54509807 0.5411765 ]
  [0.5294118  0.54509807 0.5411765 ]]

 ...

 [[0.5529412  0.53333336 0.4509804 ]
  [0.5529412  0.53333336 0.4509804 ]
  [0.54901963 0.5254902  0.44705883]
  ...
  [0.54509807 0.5529412  0.5568628 ]
  [0.54509807 0.5529412  0.56078434]
  [0.54509807 0.5529412  0.56078434]]

 [[0.54901963 0.5254902  0.44705883]
  [0.54901963 0.5254902  0.44705883]


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Label encode the y data
y_encoder = LabelEncoder().fit(y)
y = y_encoder.transform(y)

# Convert values to numpy arrays
X = np.array(X)

# Split the training dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Define the augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),         # Random rotation (20 degrees)
    tf.keras.layers.RandomTranslation(0.1, 0.1), # Random horizontal and vertical shift
    tf.keras.layers.RandomZoom(0.2),             # Random zoom
    tf.keras.layers.RandomFlip('horizontal')     # Random horizontal flip
])

In [5]:
# Create an empty list for X and y augmentations
X_train_aug = []
y_train_aug = []

# Loop through the entire X_train set
for i in range(len(X_train)):
    # Select the original image and its y label
    img = X_train[i]
    label = y_train[i]

    # Ensure that the input data has the correct shape
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Add 5 new images for every original
    for j in range(5):
        # Create and append the image
        X_train_aug.append(data_augmentation(img, training=True)[0].numpy())
        # Append the original label
        y_train_aug.append(label)

# Print the length of the augmented images and the labels
print(len(X_train_aug))
print(len(y_train_aug))

1000
1000


In [6]:
# Export our final variables to a pickle file using a dictionary
from google.colab import drive
drive.mount('/content/drive')

# Create the dictionary
fungi_dict = {
    'X_train': X_train_aug,
    'X_test': X_test,
    'y_train': y_train_aug,
    'y_test': y_test
}

# Store the dictionary in a pickle file
with open('/content/drive/My Drive/fungi_dict.pkl', 'wb') as file:
    pickle.dump(fungi_dict, file)

Mounted at /content/drive
